In [1]:
import keras
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
filepath = keras.utils.get_file(
    fname='shakespeare.txt', origin='https://homl.info/shakespeare'
)

In [3]:
with open(filepath) as file_:
    shakespeare = file_.read()

In [4]:
print(shakespeare[:20])

First Citizen:
Befor


In [5]:
tokenizar = tf.keras.preprocessing.text.Tokenizer(
    char_level=True,
    lower     =False, # Todas as palavras no formato original delas
    )

> `char_level=True` codificação a nivel de caractere em vez de nivel de palavra

In [6]:
tokenizar.fit_on_texts([shakespeare])

In [7]:
tokenizar.texts_to_sequences(['First'])

[[50, 10, 8, 7, 3]]

In [8]:
tokenizar.sequences_to_texts([[50, 10, 8, 7, 3]])

['F i r s t']

In [9]:
pd.DataFrame(tokenizar.word_index, index=[0])

,,e,t,o,a,h,s,r,n,i,...,q,x,z,J,Q,Z,X,3,&,$
0,1,2,3,4,5,6,7,8,9,10,...,56,57,58,59,60,61,62,63,64,65


In [10]:
tokenizar.document_count

1

In [11]:
encoder = np.array(tokenizar.texts_to_sequences([shakespeare])) - 1
encoder

array([[49,  9,  7, ..., 20, 25, 10]])

#### Como dividir dados sequencial

In [13]:
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(1115394,), dtype=tf.int32, name=None)>

Decompondo o conjunto de dados sequencial em várias janelas

In [14]:
passos_ = 100

In [15]:
tamanho_janela = passos_ + 1

In [24]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [25]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoder[:1_000_000], length=length, shuffle=True,
                       seed=42)
valid_set = to_dataset(encoder[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoder[1_060_000:], length=length)

In [28]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=(tokenizar.vocabulary_size() - 2), output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True)
history = model.fit(train_set, validation_data=valid_set, epochs=10,
                    callbacks=[model_ckpt])

AttributeError: 'Tokenizer' object has no attribute 'vocabulary_size'